In [2]:
import numpy as np
import json

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

import torch
torch.cuda.set_device(1)

In [66]:
def clean_transcript(text):
    text = text.replace('``', "'")
    text = text.replace('`', "'")
    text = text.replace("''", "'")
    text = text.replace(" '", "'")
    return text

with open("./evidence.json", "r") as f:
    evidences = json.load(f)
    
for evidence_id, evidence_text in evidences.items():
    evidences[evidence_id] = clean_transcript(evidence_text)
    
corpus = list(evidences.values())
evidences = tuple(evidences.items())

In [24]:
embedding_model = SentenceTransformer("all-MiniLM-L12-v2", device="cuda:1")
corpus_embeddings =embedding_model.encode(corpus, convert_to_tensor=True)

In [78]:
from sentence_transformers import SentenceTransformer, CrossEncoder

def retrieve_evidence(
    query, corpus, corpus_embeddings, embedding_model, evidence_items, top_k=50
):
    """Retrieves the top_k most similar evidence items for a given query."""
    if not embedding_model or corpus_embeddings is None:
        print("Embedding model or corpus embeddings not available.")
        return []

    query = clean_transcript(query)
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)

    # Calculate cosine similarity
    similarity_scores = cos_sim(query_embedding, corpus_embeddings)[0]

    # Get top_k scores and indices
    actual_top_k = min(top_k, len(corpus))
    scores, indices = torch.topk(similarity_scores, k=actual_top_k)

    # Map indices back to evidence items
    most_relevant_evidences = [
        (evidence_items[idx.item()], scores[i].item()) for i, idx in enumerate(indices)
    ]
    return most_relevant_evidences  # Return list of ((id, text), score) tuples

def rerank_evidence(
    query,
    retrieved_evidence,
    reranker_model,
):
    """Reranks retrieved evidence using a CrossEncoder model."""
    if not retrieved_evidence:
        print("No evidence provided for reranking.")
        return []

    try:
        query = clean_transcript(query)

        # Prepare input for the reranker: [query, evidence_text] pairs
        reranker_input = [
            [query, evidence[0][1]] for evidence in retrieved_evidence
        ]  # evidence is ((id, text), score)

        # Predict scores
        scores = reranker_model.predict(reranker_input, show_progress_bar=True)

        # Combine original evidence items with new scores and sort
        # evidence[0] is the original (id, text) tuple
        reranked_results = sorted(
            zip(scores, [item[0] for item in retrieved_evidence]), reverse=True
        )

        print("Reranking complete.")
        # Return list of (rerank_score, (id, text)) tuples
        return reranked_results

    except Exception as e:
        print(f"Error during reranking: {e}")
        return []  # Return empty list on error


In [ ]:
queries = [
    "[South Australia] has the most expensive electricity in the world.",
    "when 3 per cent of total annual global emissions of carbon dioxide are from humans and Australia prod\u00aduces 1.3 per cent of this 3 per cent, then no amount of emissions reductio\u00adn here will have any effect on global climate.",
    "This means that the world is now 1C warmer than it was in pre-industrial times",
]

reranker_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L12-v2')
top_k = 50
final_top_n = 5

for query in queries:
    print(f"\n--- Processing Query: {query} ---")

    # 4. Retrieve Evidence (Initial Retrieval)
    retrieved = retrieve_evidence(
        query,
        corpus,
        corpus_embeddings,
        embedding_model,
        evidences,
        top_k=top_k,
    )
    if not retrieved:
        print("No relevant evidence found during initial retrieval.")
        continue

    print(f"\nTop {len(retrieved)} Retrieved Evidence (before reranking):")
    for (evidence_id, evidence_text), score in retrieved[
        :5
    ]:  # Show top 5 retrieved
        print(
            f"  Score (Cosine Sim): {score:.4f} - {evidence_id}: {evidence_text[:100]}..."
        )  # Truncate long text

    # 5. Rerank Evidence
    # Use CPU for reranker by default as it's often efficient enough
    reranked_results = rerank_evidence(
        query, retrieved, reranker_model
    )  # List of (score, (id, text))
    if not reranked_results:
        print("Reranking failed.")
        continue

    print(f"\nTop {final_top_n} Reranked Evidence:")
    for i, (score, (evidence_id, evidence_text)) in enumerate(
        reranked_results[:final_top_n]
    ):
        print(f"  Rank {i+1} (Score: {score:.4f}) - {evidence_id}: {evidence_text}")

    print("-" * (len(query) + 20))  # Separator


--- Processing Query: [South Australia] has the most expensive electricity in the world. ---

Top 50 Retrieved Evidence (before reranking):
  Score (Cosine Sim): 0.8508 - evidence-67732: [citation needed] South Australia has the highest retail price for electricity in the country....
  Score (Cosine Sim): 0.8203 - evidence-572512: "South Australia has the highest power prices in the world"....
  Score (Cosine Sim): 0.6736 - evidence-780332: Industrialised countries such as Canada, the US, and Australia are among the highest per capita cons...
  Score (Cosine Sim): 0.6598 - evidence-1061888: Australia has one of the fastest deployment rates of renewable energy worldwide....
  Score (Cosine Sim): 0.6283 - evidence-48256: One of the most powerful power grids in the world supplies power to the state of Queensland, Austral...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Reranking complete.

Top 5 Reranked Evidence:
  Rank 1 (Score: 6.7758) - evidence-572512: "South Australia has the highest power prices in the world".
  Rank 2 (Score: 6.6325) - evidence-67732: [citation needed] South Australia has the highest retail price for electricity in the country.
  Rank 3 (Score: 2.1835) - evidence-780332: Industrialised countries such as Canada, the US, and Australia are among the highest per capita consumers of electricity in the world, which is possible thanks to a widespread electrical distribution network.
  Rank 4 (Score: 1.8560) - evidence-723533: According to a Sierra Club analysis, the US Kemper Project, which was due to be online in 2017, is the most expensive power plant ever built for the watts of electricity it will generate.
  Rank 5 (Score: 0.1382) - evidence-48256: One of the most powerful power grids in the world supplies power to the state of Queensland, Australia.
-------------------------------------------------------------------------------

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Reranking complete.

Top 5 Reranked Evidence:
  Rank 1 (Score: 3.8120) - evidence-609211: The report mentioned that this would require global net human-caused emissions of carbon dioxide (CO2) to fall by about 45% from 2010 levels by 2030, reaching "net zero" around 2050, through “rapid and far-reaching” transitions in land, energy, industry, buildings, transport, and cities.
  Rank 2 (Score: 3.5089) - evidence-215052: In 2008, countries with a Kyoto cap made up less than one-third of annual global carbon dioxide emissions from fuel combustion.
  Rank 3 (Score: 3.0580) - evidence-677627: SR15 also has modelling that shows that, for global warming to be limited to 1.5 °C, "Global net human-caused emissions of carbon dioxide (CO2) would need to fall by about 45 percent from 2010 levels by 2030, reaching'net zero' around 2050.
  Rank 4 (Score: 2.3935) - evidence-584052: Beyond 2012 if the growth rate were reduced to three percent yearly, carbon emissions in 2030 would be 28 MT, which is 7

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Reranking complete.

Top 5 Reranked Evidence:
  Rank 1 (Score: 6.5404) - evidence-694262: The planet is now 0.8 °C warmer than in pre-industrial times.
  Rank 2 (Score: 5.0996) - evidence-889933: Multiple independently produced instrumental datasets confirm that the 2009–2018 decade was 0.93 ± 0.07 °C warmer than the pre-industrial baseline (1850–1900).
  Rank 3 (Score: 4.5197) - evidence-590642: Nevertheless, the gases which have been emitted so far are unlikely to cause global temperature to rise to 1.5°C alone, meaning a global temperature rise to 1.5°C above pre-industrial levels is avoidable, assuming net zero emissions are reached soon.
  Rank 4 (Score: 4.1413) - evidence-246929: It is a major aspect of climate change, and has been demonstrated by the instrumental temperature record which shows global warming of around 1 °C since the pre-industrial period, although the bulk of this (0.9°C) has occurred since 1970.
  Rank 5 (Score: 3.8136) - evidence-1017598: Global warming will l